In [344]:
import pandas as pd 
from patsy import dmatrices
import numpy as np

In [345]:
data = pd.read_csv("NFLPlaybyPlay2015.csv", dtype={'DefTwoPoint': str}, na_values=['NA'])
data.head()

,Unnamed: 0,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,...,AcceptedPenalty,PenalizedTeam,PenaltyType,PenalizedPlayer,PenaltyYards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,Season
0,36.0,9/10/2015,2.015091e+09,1.0,1.0,NaN,15:00,15.0,3600.0,0.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2015.0
1,51.0,9/10/2015,2.015091e+09,1.0,1.0,1.0,15:00,15.0,3600.0,0.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2015.0
2,72.0,9/10/2015,2.015091e+09,1.0,1.0,1.0,14:21,15.0,3561.0,39.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2015.0
3,101.0,9/10/2015,2.015091e+09,1.0,1.0,2.0,14:04,15.0,3544.0,17.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2015.0
4,122.0,9/10/2015,2.015091e+09,1.0,1.0,1.0,13:26,14.0,3506.0,38.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2015.0


In [346]:
df = data.copy()
df.columns.values

array(['Unnamed: 0', 'Date', 'GameID', 'Drive', 'qtr', 'down', 'time',
       'TimeUnder', 'TimeSecs', 'PlayTimeDiff', 'SideofField', 'yrdln',
       'yrdline100', 'ydstogo', 'ydsnet', 'GoalToGo', 'FirstDown',
       'posteam', 'DefensiveTeam', 'desc', 'PlayAttempted', 'YardsGained',
       'sp', 'Touchdown', 'ExPointResult', 'TwoPointConv', 'DefTwoPoint',
       'Safety', 'PuntResult', 'PlayType', 'Passer', 'PassAttempt',
       'PassOutcome', 'PassLength', 'PassLocation', 'InterceptionThrown',
       'Interceptor', 'Rusher', 'RushAttempt', 'RunLocation', 'RunGap',
       'Receiver', 'Reception', 'ReturnResult', 'Returner',
       'BlockingPlayer', 'Tackler1', 'Tackler2', 'FieldGoalResult',
       'FieldGoalDistance', 'Fumble', 'RecFumbTeam', 'RecFumbPlayer',
       'Sack', 'ChallengeReplay', 'ChalReplayResult', 'AcceptedPenalty',
       'PenalizedTeam', 'PenaltyType', 'PenalizedPlayer', 'PenaltyYards',
       'PosTeamScore', 'DefTeamScore', 'ScoreDiff', 'AbsScoreDiff',
       'Season

In [347]:
#Extra Point
df['ExPointAborted'] = pd.get_dummies(df['ExPointResult'])["Aborted"]
df['ExPointMade']= pd.get_dummies(df['ExPointResult'])["Made"]
df['ExPointMissed']= pd.get_dummies(df['ExPointResult'])["Missed"]

#Two Point Conv
df['TwoPointSuc']= pd.get_dummies(df['TwoPointConv'])["Success"]
df['TwoPointFail']= pd.get_dummies(df['TwoPointConv'])["Failure"]

#Defense Two Point
df['DefTwoPointSuc']= pd.get_dummies(df['DefTwoPoint'])["Success"]
df['DefTwoPointFail']= pd.get_dummies(df['DefTwoPoint'])["Failure"]

# Punt Result
df['PuntResultClean']= pd.get_dummies(df['PuntResult'])["Clean"]
df['PuntResultBlocked']= pd.get_dummies(df['PuntResult'])["Blocked"]

#Play Type
df['KickoffPlay']= pd.get_dummies(df['PlayType'])["Kickoff"]
df['RunPlay']= pd.get_dummies(df['PlayType'])["Run"]
df['PassPlay']= pd.get_dummies(df['PlayType'])["Pass"]
df['SackPlay']= pd.get_dummies(df['PlayType'])["Sack"]
df['NoPlay']= pd.get_dummies(df['PlayType'])["No Play"]
df['FieldGoalPlay']= pd.get_dummies(df['PlayType'])["Field Goal"]
df['PuntPlay']= pd.get_dummies(df['PlayType'])["Punt"]
df['ExtraPointPlay']= pd.get_dummies(df['PlayType'])["Extra Point"]
df['QBKneelPlay']= pd.get_dummies(df['PlayType'])["QB Kneel"]
df['SpikePlay']= pd.get_dummies(df['PlayType'])["Spike"]
df['OnsideKickPlay']= pd.get_dummies(df['PlayType'])["Onside Kick"]

#Pass Outcome
df['CompletedPass']= pd.get_dummies(df['PassOutcome'])["Complete"]
df['IncompletePass']= pd.get_dummies(df['PassOutcome'])["Incomplete Pass"]

#Side of Field
df["OffenseSide"] = (df["SideofField"] == df["posteam"])

#Pass Length
df["ShortPass"] = pd.get_dummies(df['PassLength'])["Short"]
df["DeepPass"] = pd.get_dummies(df['PassLength'])["Deep"]

#Pass Location
df["LeftPass"]= pd.get_dummies(df['PassLocation'])["left"]
df["MiddlePass"]= pd.get_dummies(df['PassLocation'])["middle"]
df["RightPass"]= pd.get_dummies(df['PassLocation'])["right"]

#Run Location
df["LeftRun"]= pd.get_dummies(df['RunLocation'])["left"]
df["MiddleRun"]= pd.get_dummies(df['RunLocation'])["middle"]
df["RightRun"]= pd.get_dummies(df['RunLocation'])["right"]

#Run Gap
df["RunGapEnd"]= pd.get_dummies(df['RunGap'])["end"]
df["RunGapGuard"]= pd.get_dummies(df['RunGap'])["guard"]
df["RunGapTackle"]= pd.get_dummies(df['RunGap'])["tackle"]

#Return Result
df["ReturnResultTD"]= pd.get_dummies(df['ReturnResult'])["Touchdown"]
df["ReturnResultTouch"]= pd.get_dummies(df['ReturnResult'])["Touchback"]

#Offense Fumble Recovery
df["OffFumbleRecover"] = (df["RecFumbTeam"] == df["posteam"])

#Challenge Result
df["ChalOverturned"]= pd.get_dummies(df['ChalReplayResult'])["Reversed"]
df["ChalAccept"]= pd.get_dummies(df['ChalReplayResult'])["Upheld"]

#Penalty
df["OffPenalty"] = (df["PenalizedTeam"] == df["posteam"])

#Return Result
df["Touchback"]= pd.get_dummies(df['ReturnResult'])["Touchback"]
df["FairCatch"]= pd.get_dummies(df['ReturnResult'])["Fair Catch"]

In [348]:
df = df.drop(df[df['PlayType'] == "Quarter End"].index)
df = df.drop(df[df['PlayType'] == "Two Minute Warning"].index)
df = df.drop(df[df['PlayType'] == "Timeout"].index)
df = df.drop(df[df['PlayType'] == "End of Game"].index)
df = df.drop(df[df['PlayType'] == "Half End"].index)

In [349]:
df = df.drop(['Unnamed: 0','time','TimeSecs','yrdln','desc','ExPointResult',
               'TwoPointConv','SideofField','DefensiveTeam','DefTwoPoint','Safety',
               'PuntResult','PassAttempt','PassOutcome','Rusher', 'Season',
               'RushAttempt','ReturnResult','ExPointMade','ExPointMissed'], axis=1)

df = df.drop(['Receiver','Reception','InterceptionThrown','Interceptor','Receiver','Reception',
               'Returner','BlockingPlayer','Tackler1','Tackler2','FieldGoalResult','FieldGoalDistance',
               'Fumble','RecFumbTeam','RecFumbPlayer','Sack','ChallengeReplay','ChalReplayResult',
               'AcceptedPenalty','PenalizedTeam','PenaltyType','PenalizedPlayer','PenaltyYards',
               'posteam','OffFumbleRecover'], axis =1)

df = df.drop(['Passer','PlayType','RunLocation','RunGap','PassLocation','PassLength',
             'ExPointAborted', 'TwoPointSuc','TwoPointFail', 'DefTwoPointSuc',
             'DefTwoPointFail','GameID', 'PuntResultClean', 'PuntResultBlocked',
             'KickoffPlay','NoPlay',
             'FieldGoalPlay', 'PuntPlay', 'ExtraPointPlay', 'QBKneelPlay',
             'SpikePlay', 'OnsideKickPlay','ReturnResultTD', 'ReturnResultTouch',
             'ChalAccept', 'OffPenalty','Date','FirstDown','PlayAttempted'], axis=1)

df = df.drop(['ydsnet','YardsGained','ChalOverturned','sp','OffenseSide'], axis =1)

In [350]:
df.columns.values

array(['Drive', 'qtr', 'down', 'TimeUnder', 'PlayTimeDiff', 'yrdline100',
       'ydstogo', 'GoalToGo', 'Touchdown', 'PosTeamScore', 'DefTeamScore',
       'ScoreDiff', 'AbsScoreDiff', 'RunPlay', 'PassPlay', 'SackPlay',
       'CompletedPass', 'IncompletePass', 'ShortPass', 'DeepPass',
       'LeftPass', 'MiddlePass', 'RightPass', 'LeftRun', 'MiddleRun',
       'RightRun', 'RunGapEnd', 'RunGapGuard', 'RunGapTackle', 'Touchback',
       'FairCatch'], dtype=object)

In [351]:
df['LastRunPlay'] = df.RunPlay.shift(1)
df['LastPassPlay'] = df.PassPlay.shift(1)
df['LastSackPlay'] = df.SackPlay.shift(1)
df['LastCompletedPass'] = df.CompletedPass.shift(1)
df['LastIncompletePass'] = df.IncompletePass.shift(1)
df['LastShortPass'] = df.ShortPass.shift(1)
df['LastDeepPass'] = df.DeepPass.shift(1)
df['LastLeftPass'] = df.LeftPass.shift(1)
df['LastMiddlePass'] = df.MiddlePass.shift(1)
df['LastRightPass'] = df.RightPass.shift(1)
df['LastLeftRun'] = df.LeftRun.shift(1)
df['LastMiddleRun'] = df.MiddleRun.shift(1)
df['LastRightRun'] = df.RightRun.shift(1)
df['LastRunGapEnd'] = df.RunGapEnd.shift(1)
df['LastRunGapGuard'] = df.RunGapGuard.shift(1)
df['LastRunGapTackle'] = df.RunGapTackle.shift(1)
df['LastTouchback'] = df.Touchback.shift(1)
df['LastFairCatch'] = df.FairCatch.shift(1)

In [352]:
df['y'] = np.where(df['Touchdown'] == 1.0, 1, 0) 

In [353]:
df = df.drop(['SackPlay','CompletedPass','IncompletePass','ShortPass','DeepPass','LeftPass',
             'MiddlePass', 'RightPass',
             'Touchback','FairCatch','Touchdown','PassPlay','RunPlay'], axis=1)

In [354]:
df.columns.values

array(['Drive', 'qtr', 'down', 'TimeUnder', 'PlayTimeDiff', 'yrdline100',
       'ydstogo', 'GoalToGo', 'PosTeamScore', 'DefTeamScore', 'ScoreDiff',
       'AbsScoreDiff', 'LeftRun', 'MiddleRun', 'RightRun', 'RunGapEnd',
       'RunGapGuard', 'RunGapTackle', 'LastRunPlay', 'LastPassPlay',
       'LastSackPlay', 'LastCompletedPass', 'LastIncompletePass',
       'LastShortPass', 'LastDeepPass', 'LastLeftPass', 'LastMiddlePass',
       'LastRightPass', 'LastLeftRun', 'LastMiddleRun', 'LastRightRun',
       'LastRunGapEnd', 'LastRunGapGuard', 'LastRunGapTackle',
       'LastTouchback', 'LastFairCatch', 'y'], dtype=object)

In [363]:
df

,Drive,qtr,down,TimeUnder,PlayTimeDiff,yrdline100,ydstogo,GoalToGo,PosTeamScore,DefTeamScore,...,LastRightPass,LastLeftRun,LastMiddleRun,LastRightRun,LastRunGapEnd,LastRunGapGuard,LastRunGapTackle,LastTouchback,LastFairCatch,y
0,1.0,1.0,NaN,15.0,0.0,35.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1.0,1.0,1.0,15.0,0.0,80.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,1.0,1.0,1.0,15.0,39.0,62.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0
3,1.0,1.0,2.0,15.0,17.0,53.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.0,1.0,1.0,14.0,38.0,49.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
5,1.0,1.0,1.0,13.0,44.0,35.0,10.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,1.0,1.0,1.0,13.0,37.0,24.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
7,1.0,1.0,2.0,12.0,45.0,32.0,18.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,1.0,1.0,2.0,11.0,27.0,42.0,28.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,1.0,1.0,3.0,11.0,25.0,36.0,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0


In [355]:
s = ""
for i in df.columns.values: 
    s += i + " + " 
len(s) 
s = s[0:len(s)-6] 
s += "+ 0" 

s

'Drive + qtr + down + TimeUnder + PlayTimeDiff + yrdline100 + ydstogo + GoalToGo + PosTeamScore + DefTeamScore + ScoreDiff + AbsScoreDiff + LeftRun + MiddleRun + RightRun + RunGapEnd + RunGapGuard + RunGapTackle + LastRunPlay + LastPassPlay + LastSackPlay + LastCompletedPass + LastIncompletePass + LastShortPass + LastDeepPass + LastLeftPass + LastMiddlePass + LastRightPass + LastLeftRun + LastMiddleRun + LastRightRun + LastRunGapEnd + LastRunGapGuard + LastRunGapTackle + LastTouchback + LastFairCatch + 0'

In [356]:
formula = 'y ~' + s 

Y,X = dmatrices(formula, df, return_type = 'dataframe')

In [357]:
y =Y['y'].values

In [358]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [359]:
result = model.fit(X_train, y_train)

In [360]:
from sklearn import metrics

prediction_train = model.predict(X_train)
print metrics.accuracy_score(y_train, prediction_train)

0.96655922643


In [361]:
prediction = model.predict(X_test)
print metrics.accuracy_score(y_test, prediction)

0.965988719877


In [362]:
weights = pd.Series(model.coef_[0],
                 index=X.columns.values)
weights.sort_values(ascending = False) 

GoalToGo              0.923025
LastSackPlay          0.241668
LastRunGapTackle      0.240434
LastRunPlay           0.157959
LastRightPass         0.121391
LastShortPass         0.118545
LastMiddleRun         0.103991
PosTeamScore          0.074062
DefTeamScore          0.051618
LastDeepPass          0.049921
LastRightRun          0.045403
LastLeftPass          0.033434
TimeUnder             0.032338
ScoreDiff             0.022444
LastMiddlePass        0.013641
AbsScoreDiff          0.010086
RunGapEnd             0.000862
LastIncompletePass   -0.002663
PlayTimeDiff         -0.003036
LastLeftRun          -0.039494
yrdline100           -0.045676
ydstogo              -0.061480
Drive                -0.061848
LastRunGapEnd        -0.069597
LastCompletedPass    -0.080893
LastFairCatch        -0.148435
LastPassPlay         -0.199006
down                 -0.233832
LastRunGapGuard      -0.323774
LeftRun              -0.327586
LastTouchback        -0.364351
RunGapGuard          -0.417717
RunGapTa